### Import required libraries

In [ ]:
from kubernetes.client import V1PodTemplateSpec
from kubernetes.client import V1ObjectMeta
from kubernetes.client import V1PodSpec
from kubernetes.client import V1Container

from kubeflow.tfjob import constants
from kubeflow.tfjob import utils
from kubeflow.tfjob import V1ReplicaSpec
from kubeflow.tfjob import V1TFJob
from kubeflow.tfjob import V1TFJobSpec
from kubeflow.tfjob import TFJobClient

In [ ]:
namespace = utils.get_default_target_namespace()

### Define TFJob

In [ ]:
container = V1Container(
    name="tensorflow",
    image="gcr.io/gsoc-kf-example/tf_2_text_classification:1.4",
    args=[
        "--learning_rate=0.01",
        "--epochs=1",
        "--batch_size=64"
        ]
)

worker = V1ReplicaSpec(
    replicas=1,
    restart_policy="OnFailure",
    template=V1PodTemplateSpec(
        spec=V1PodSpec(
            containers=[container]
        )
    )
)

chief = V1ReplicaSpec(
    replicas=1,
    restart_policy="OnFailure",
    template=V1PodTemplateSpec(
        spec=V1PodSpec(
            containers=[container]
        )
    )
)

ps = V1ReplicaSpec(
    replicas=1,
    restart_policy="OnFailure",
    template=V1PodTemplateSpec(
        spec=V1PodSpec(
            containers=[container]
        )
    )
)

tfjob = V1TFJob(
    api_version="kubeflow.org/v1",
    kind="TFJob",
    metadata=V1ObjectMeta(name="text-classification",namespace=namespace),
    spec=V1TFJobSpec(
        clean_pod_policy="All",
        tf_replica_specs={"Worker": worker,
                          "Chief": chief,
                          "PS": ps}
    )
)

### Create TFJob

In [ ]:
tfjob_client = TFJobClient()
tfjob_client.create(tfjob, namespace=namespace)

### Get the created TFJob

In [ ]:
tfjob_client.get('text-classification', namespace=namespace)

### Get the TFJob status, check if the TFJob has been started

In [ ]:
tfjob_client.get_job_status('text-classification', namespace=namespace)

### Wait for the specified job to finish

In [ ]:
tfjob_client.wait_for_job('text-classification', namespace=namespace, watch=True, timeout_seconds=7200)

### Check if the TFJob succeeded

In [ ]:
tfjob_client.is_job_succeeded('text-classification', namespace=namespace)

### Get the TFJob training logs

In [ ]:
tfjob_client.get_logs('text-classification', namespace=namespace)

### Delete the TFJob

In [ ]:
tfjob_client.delete('text-classification', namespace=namespace)